# MHKiT WEC-Sim Example

This notebook demonstrates using the MHKiT wave module and [WEC-Sim](http://wec-sim.github.io/WEC-Sim/index.html) together to perform a resource characterization study in MHKiT, simulate representative cases with WEC-Sim, and visualize the results in MHKiT to estimate MAEP (Mean Annual Energy Production).

  1. Characterize the available resource at a location
     - See the PacWave example notebook
  2. Write a WEC-Sim batch file for the given clusters
  3. Simulate the device *in WEC-Sim*.
     - Ensure that the spectra used in WEC-Sim is identical to the one used in MHKiT.
  4. Load WEC-Sim batch results
  5. Assess results and visualize quantities of interest

This example uses WEC-Sim to simulate the [Oscillating Surge Wave Energy Converter (OSWEC)](https://wec-sim.github.io/WEC-Sim/main/user/tutorials.html#oscillating-surge-wec-oswec), a flap-type device.

Start by importing MHKiT and the necessary python packages (e.g.`scipy.io`, `matplotlib.pyplot`, `pandas`, `numpy`).

In [1]:
from mhkit import wave
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'mhkit'

## 1. Characterize the available resource at a location

This example will use an abbreviated version of `PacWave_resource_characterization_example.ipynb`. 
For full details on downloading, calculating, and visualizing the k-means clusters representation of the site's wave resouce, see that example.

We select the N=32 cluster as it's total energy flux is closet to the total energy flux of the site considering all wave conditions. We will load the PacWave example output, which can be easily saved after running the example with the command `results[32].to_csv("pacwave_cluster_32.csv")`. We will start this example by reading in that csv output and formatting it for WEC-Sim.

In [2]:
results = pd.read_csv("data/wave/pacwave_cluster_32.csv", index_col=0)
results

NameError: name 'pd' is not defined

## 2. Write a WEC-Sim batch file for the given clusters

[WEC-Sim MCR](https://wec-sim.github.io/WEC-Sim/main/user/advanced_features.html#multiple-condition-runs-mcr) (multiple condition run) files should contain a structure `mcr` that contains two variables: `header` and `cases`. Each column of `header` and `cases` denotes a variable and it's value respectively. Each row is another simulation. WEC-Sim defines waves using the significant wave height and peak period. We will isolate these values from the results of the cluster analysis and create a dictionary that is written to the `.mat` file.

In [3]:
ws_mcr_cases = results[["Hm0","Tp"]]
ws_mcr_header = np.array(["waves.height","waves.period"], dtype='object')
ws_mcr_out = {'mcr': {'header': ws_mcr_header, 'cases': ws_mcr_cases}}
sio.savemat('mcr_mhkit.mat', ws_mcr_out)

NameError: name 'results' is not defined

## 3. Simulate the device *in WEC-Sim*

Now that the MCR file is created, we need to go simulate WEC performance in these wave conditions using WEC-Sim. To recreate the data used in the next step, use the created MCR file with WEC-Sim's [OSWEC example](https://github.com/WEC-Sim/WEC-Sim/tree/main/examples/OSWEC). For an accurate comparison to the power calculated in the resource characterization, we should ensure that the WEC-Sim cases use irregular JONSWAP wave spectra as in the PacWave example.

For convenience in this demonstration, we enforce OSWEC model stability in the extreme wave conditions by arbitrarily applying a large PTO stiffness and damping:
```
pto(1).stiffness = 1e5;
pto(1).damping = 5e7;
```

To reduce the amount of extranenous data saved for this example, we limit the WEC-Sim output to the PTO's power output in the `userDefinedFunctions.m` script:
```
if exist('imcr','var')
    if imcr == 1
        nmcr = size(mcr.cases,1);
        power = nan(1, nmcr);
    end

    iRampEnd = simu.rampTime./simu.dtOut + 1;
    power(imcr) = -mean(output.ptos(1).powerInternalMechanics(iRampEnd:end,5));

    if imcr == nmcr
        % Save output
        save('mcr_mhkit_power.mat', 'power');
    end
end
```

## 4. Load WEC-Sim batch results

Note that in this example we do not save the entire WEC-Sim `output` structure for each case. See the `wecsim_example.ipynb` for information on loading that WEC-Sim data. Here the output is one array of average power output that we will load and compare to the resource characterization.

Note that the power output \[W\] is significantly larger than the energy flux \[W/m\] due to the width of the OSWEC. 

In [4]:
# Relative location and filename of simulated WEC-Sim data (run with mooring)
filename = "./data/wave/mcr_mhkit_power.mat"

# Load data using the `wecsim.read_output` function which returns a dictionary of Datasets.
wecsim_data = sio.loadmat(filename)
results['P'] = wecsim_data['power'][0]
results

NameError: name 'sio' is not defined

## 5. Assess results and visualize quantities of interest
Now that we have loaded the OSWEC's modeled power, we can assess it's performance relative to the incoming wave and calculate the mean annual energy production (MAEP) using MHKiT.

In [5]:
results['CW'] = wave.performance.capture_width(results['P'], results['J'])
oswec_width = 18
results['CWR'] = results['CW'] / oswec_width
results

NameError: name 'wave' is not defined

In [6]:
MAEP = wave.performance.mean_annual_energy_production_matrix(results['CW'], results['J'], results['weights']) / 1000 # kWh
MAEP = np.round(MAEP, 0).item()
MAEP

NameError: name 'wave' is not defined